In [3]:
import random

class RSA:

    def __init__(self, key_size: int = 1024):
        self.key_size = key_size
        self.public_key = None
        self.private_key = None

    def _modular_exponentiation(self, base: int, exp: int, mod: int) -> int:
        """Tính (base^exp) % mod (Square-and-Multiply)"""
        result = 1
        base = base % mod
        while exp > 0:
            if exp % 2 == 1:
                result = (result * base) % mod
            base = (base * base) % mod
            exp = exp // 2
        return result

    def _extended_gcd(self, a: int, b: int):
        """Euclid mở rộng: Trả về (gcd, x, y)"""
        old_r, r = a, b
        old_x, x = 1, 0
        old_y, y = 0, 1
        while r != 0:
            quotient = old_r // r
            old_r, r = r, old_r - quotient * r
            old_x, x = x, old_x - quotient * x
            old_y, y = y, old_y - quotient * y
        return old_r, old_x, old_y

    def _modular_inverse(self, a: int, m: int) -> int:
        """Nghịch đảo modulo"""
        gcd, x, y = self._extended_gcd(a, m)
        if gcd != 1:
            raise ValueError("Không tồn tại nghịch đảo")
        return x % m

    def _is_prime_miller_rabin(self, n: int, k: int = 40) -> bool:
        """Kiểm tra nguyên tố Miller-Rabin"""
        if n == 2 or n == 3: return True
        if n % 2 == 0 or n < 2: return False

        r, d = 0, n - 1
        while d % 2 == 0:
            r += 1
            d //= 2
        
        for _ in range(k):
            a = random.randint(2, n - 2)
            x = self._modular_exponentiation(a, d, n)
            if x == 1 or x == n - 1:
                continue
            for _ in range(r - 1):
                x = self._modular_exponentiation(x, 2, n)
                if x == n - 1:
                    break
            else:
                return False
        return True

    def _generate_prime(self, bits: int) -> int:
        """Sinh số nguyên tố n-bit"""
        while True:
            # Đảm bảo bit đầu và cuối là 1
            num = random.getrandbits(bits)
            num |= (1 << (bits - 1)) | 1 
            if self._is_prime_miller_rabin(num):
                return num

    def generate_keys(self):
        print(f"[*] Đang sinh khóa {self.key_size} bit")
        
        # 1. Sinh p, q (Mỗi số chiếm 1 nửa độ dài)
        p = self._generate_prime(self.key_size // 2)
        q = self._generate_prime(self.key_size // 2)
        while p == q:
            q = self._generate_prime(self.key_size // 2)

        # 2. Tính n, phi
        n = p * q
        phi_n = (p - 1) * (q - 1)

        # 3. Chọn e (Chuẩn công nghiệp là 65537)
        e = 65537
        while self._extended_gcd(e, phi_n)[0] != 1:
            e = random.randrange(3, phi_n, 2)

        # 4. Tính d
        d = self._modular_inverse(e, phi_n)

        self.public_key = (e, n)
        self.private_key = (d, n)
        print(f" Sinh khóa thành công!")

    def encrypt_message(self, text: str) -> list[int]:
        """Mã hóa văn bản dài thành danh sách các số nguyên"""
        if not self.public_key: raise Exception("Thiếu Public Key!")
        e, n = self.public_key
        
        # 1. Chuyển Text -> Bytes
        msg_bytes = text.encode('utf-8')
        
        # 2. Tính kích thước khối (Block Size)
        # n có độ dài bit_length. Chia 8 để ra số bytes.
        # Trừ 1 để đảm bảo giá trị số của khối (M) luôn < n
        k_bytes = (n.bit_length() + 7) // 8
        block_size = k_bytes - 1
        
        encrypted_blocks = []
        
        # 3. Cắt và Mã hóa từng khối
        for i in range(0, len(msg_bytes), block_size):
            # Lấy 1 đoạn bytes
            chunk = msg_bytes[i : i + block_size]
            
            # Bytes -> Int (Big Endian: byte đầu tiên là quan trọng nhất)
            m_int = int.from_bytes(chunk, byteorder='big')
            
            # Mã hóa: C = m^e mod n
            c_int = self._modular_exponentiation(m_int, e, n)
            encrypted_blocks.append(c_int)
            
        return encrypted_blocks

    def decrypt_message(self, cipher_blocks: list[int]) -> str:
        """Giải mã danh sách số nguyên thành văn bản"""
        if not self.private_key: raise Exception("Thiếu Private Key!")
        d, n = self.private_key
        
        decrypted_bytes = b""
        
        for c_int in cipher_blocks:
            # Giải mã: M = c^d mod n
            m_int = self._modular_exponentiation(c_int, d, n)
            
            # Int -> Bytes
            # Tính độ dài byte cần thiết để chứa số m_int
            length = (m_int.bit_length() + 7) // 8
            chunk = m_int.to_bytes(length, byteorder='big')
            
            decrypted_bytes += chunk
            
        # Bytes -> Text
        return decrypted_bytes.decode('utf-8')

# --- TEST DRIVE (THỰC HÀNH) ---
if __name__ == "__main__":
    # 1. Khởi tạo (Key 1024 bit)
    rsa_system = RSA(key_size=1024) 
    rsa_system.generate_keys()
    
    # 2. Kịch bản Chat
    # Thay đổi tin nhắn thành "giaothongvantai"
    secret_msg = "giaothongvantai"
    print(f"\ntin nhắn: '{secret_msg}'")
    
    # 3. Mã hóa
    cipher_data = rsa_system.encrypt_message(secret_msg)
    print(f"\nGói tin bị mã hóa (gồm {len(cipher_data)} khối):")
    # In ra một phần của khối mã hóa đầu tiên để xem
    print(f"   Block 1: {str(cipher_data[0])[:30]}...") 
        
    # 4. Giải mã
    received_msg = rsa_system.decrypt_message(cipher_data)
    print(f"\ngiải mã: '{received_msg}'")
    
    # 5. Kiểm tra toàn vẹn
    if secret_msg == received_msg:
        print("\n=>Tin nhắn nguyên vẹn. ")
    else:
        print("\n=>Tin nhắn bị sai lệch. ")


[*] Đang sinh khóa 1024 bit
 Sinh khóa thành công!

tin nhắn: 'giaothongvantai'

Gói tin bị mã hóa (gồm 1 khối):
   Block 1: 993646766486068628059996434669...

giải mã: 'giaothongvantai'

=>Tin nhắn nguyên vẹn. 


In [10]:
class RSA:
    
    def __init__(self, key_size: int = 1024):
        self.key_size = key_size
        self.public_key = None
        self.private_key = None
#tinh luy thua cua modulo    
    def _modular_exponentiation(self, base: int, exp: int, mod: int )-> int:
        result = 1
        base =  base % mod

        while exp > 0:
            
            if exp % 2 == 1:
                result = (result * base) % mod
            
            # Bình phương cơ số (Square)
            base = (base * base) % mod
            
            ## Dịch bit sang phải 
            exp = exp // 2
        return result
    
    # khoa d tu khoa cong khai, sử dụng thuật toán Euclid mở rộng (EEA) để tìm hệ số x trong phương trình a + by = gcd(a, b). Nếu GCD = 1, thì x chính là nghịch đảo.
    def  _extended_gcd(self, a: int, b: int):
        """Trả về (gcd, x, y)"""
        old_r, r = a, b
        old_x, x = 1, 0
        old_y, y = 0, 1
        
        while r != 0:
            quotient = old_r // r
            # Cập nhật song song (Pythonic way)
            old_r, r = r, old_r - quotient * r
            old_x, x = x, old_x - quotient * x
            old_y, y = y, old_y - quotient * y
            
        return old_r, old_x, old_y
    
    #tim d sao cho ( a *d ) % m ==1
    def _modular_inverse(self, a:int , m:int) -> int:
        gcd, x, y =self._extended_gcd(a, m)
        if gcd != 1:
            raise ValueError('khong ton tai nghich dao')
        return x % m # ket qua duong
    
    def _is_prime_miller_rabin(self, n: int, k: int = 40) ->bool:
        
        #kiem tra n co phai so nguyen to hay khong
        # k = 40 la tieu chuan li thuyet
        
        #1. cac truong hop co ban
        if n == 2 or n ==3: return True
        if n % 2 == 0 or n < 2: return False
        
        # 2. Phân tích n-1 = d * 2^r
        # Mục đích: Tách phần lẻ (d) và phần lũy thừa 2 (r)
        r, d = 0, n -1
        while d % 2== 0:
            r += 1
            d //= 2
        
        for _ in range(k):
            #Chọn ngẫu nhiên số a trong khoảng [2, n-2]
            a = random.randint(2, n -2)
            
            #tinh = a^d % n 
            x = self._modular_exponentiation(a, d, n)
            
            if x == 1 or x == n -1:
                continue
            
            #Bình phương r-1 lần để xem x có trở thành n-1 không
            for _ in range(r -1):
                x = self._modular_exponentiation(x, 2, n)
                if x == n-1:
                    break # Thoát vòng lặp bình phương
                else:
                # Nếu chạy hết vòng lặp mà không break -> Chắc chắn là Hợp số
                    return False
            
            return True # Vượt qua k vòng thử thách -> Xác suất cao là Nguyên tố
        
        #sinh 1 so ngau nhien co do dai bit 
        def _generate_prime(self, bits: int) -> int:
            
            while True:
                # random.getrandbits Trả về một số nguyên x được tạo thành từ k bit ngẫu nhiên                                                                                      
                num = random.getrandbits(bits)
        
                # 2. Đảm bảo bit cao nhất là 1 (để đủ độ dài bits) 
                # và bit thấp nhất là 1 (để là số lẻ) (0110 -> 1111)
                num |= (1 << (bits - 1)) | 1
                if self._is_prime_miller_rabin(num):
                    return num
        
        #Sinh cặp khóa Public (e, n) và Private (d, n).
        def generate_key(self):
            
            # 1. Sinh p, q. Mỗi số có độ dài bằng một nửa tổng độ dài khóa
            p = self._generate_prime(self.key_size // 2)
            q = self._generate_prime(self.key_size // 2)
            
            #p và q khác nhau 
            while q == p:
                q = self._generate_prime(self.key_size // 2)
            
            # tinh n, phi(n)
            n = p * q
            phi_n = (p -1) * (q -1)
            
            #thay vi chon e ngau nhien, e cong nghiep se tot hon
            e = 65537
            
            # Kiểm tra điều kiện nguyên tố cùng nhau (GCD = 1)
            # Nếu xui xẻo GCD != 1, ta chọn e ngẫu nhiên khác (dự phòng)
            while self._extended_gcd(e, phi_n)[0] != 1:
                e = random.randrange(3, phi_n, 2) # random( bat dau : 3, ket thuc: phi(n), buoc: 2)
             
            #tinh private key d   
            d = self._modular_inverse(e, phi_n)
            
            #luu lai cac khoa
            self.public_key = (e, n)
            self.private_key = (d, n)
            
            print(f"Sinh khóa thành công!")
            print(f"Public Key (e, n): e=65537, n={str(n)[:10]}...")
            print(f"Private Key (d, n): d={str(d)[:10]}...")
        
        #giai ma    
        def decrypt(self, ciphertext_int: int) -> int:
            
            if self.private_key is None:
                raise Exception("Lỗi: Chưa có khóa Private!")
            
            d,n = self.private_key
            return self._modular_exponentiation(ciphertext_int, d, n)
            
                    